# PyG explainability

In [4]:
from utils.models import model_selector
from utils.datasets import load_dataset, parse_config

from torch_geometric.data import Data
from torch_geometric.explain import Explainer, GNNExplainer

import os
GNN_MODEL = "GNN"
DATASET = "syn1"
cfg_path = f"./configs/{GNN_MODEL}/{DATASET}.json"
cfg = parse_config(config_path=cfg_path)

No config found


In [ ]:
# A homogenous graph data object.
# data = Data(...)
dataset, _ = load_dataset(dataset=DATASET)
data = dataset[0]

# instantiate GNN model
model, _ = model_selector(paper=GNN_MODEL, dataset=DATASET, pretrained=False, config=cfg)


explainer = Explainer(
    model=model,
    algorithm=GNNExplainer(epochs=200),
    explanation_type='model',
    node_mask_type='attributes',
    edge_mask_type='object',
    model_config=dict(
        mode='multiclass_classification',
        task_level='node',
        return_type='log_probs',  # Model returns log of probability.
    ),
)

# Generate explanation for the node at index `10`:
explanation = explainer(data.x, data.edge_index, index=10)
print(explanation.edge_mask)
print(explanation.node_mask)